## Setup and Import Libraries

In [2]:
import os
import chromadb
from utils import word_wrap
from pypdf import PdfReader
from openai import OpenAI
from langchain_text_splitters import (
    RecursiveCharacterTextSplitter,
    SentenceTransformersTokenTextSplitter
)
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction
from chromadb.config import DEFAULT_TENANT, DEFAULT_DATABASE, Settings
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [4]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [7]:
openai_client = OpenAI()

In [8]:
chroma_client = chromadb.Client()

## Reading from PDF

In [9]:
reader = PdfReader("data/microsoft_annual_report_2022.pdf")

In [10]:
pdf_texts = [p.extract_text().strip() for p in reader.pages]

In [11]:
pdf_texts = [text for text in pdf_texts if text]

In [12]:
print(word_wrap(pdf_texts[0]))

1 
Dear shareholders, colleagues, customers, and partners:  
We are
living through a period of historic economic, societal, and
geopolitical change. The world in 2022 looks nothing like 
the world in
2019. As I write this, inflation is at a 40 -year high, supply chains
are stretched, and the war in Ukraine is 
ongoing. At the same time, we
are entering a technological era with the potential to power awesome
advancements 
across every sector of our economy and society. As the
world’s largest software company, this places us at a historic

intersection of opportunity and responsibility to the world around us.
 
Our mission to empower every person and every organization on the
planet to achieve more has never been more 
urgent or more necessary.
For all the uncertainty in the world, one thing is clear: People and
organizations in every 
industry are increasingly looking to digital
technology to overcome today’s challenges and emerge stronger. And no

company is better positioned to help t

## Applying Text Splitting

### RecursiveCharacterTextSplitter

In [13]:
character_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", ". ", " ", ""],
    chunk_size=1000,
    chunk_overlap=0
)

character_split_texts = character_splitter.split_text('\n\n'.join(pdf_texts))

print(word_wrap(character_split_texts[10]))
print(f"\nTotal chunks: {len(character_split_texts)}")

increased, due in large part to significant global datacenter
expansions and the growth in Xbox sales and usage. Despite 
these
increases, we remain dedicated to achieving a net -zero future. We
recognize that progress won’t always be linear, 
and the rate at which
we can implement emissions reductions is dependent on many factors that
can fluctuate over time.  
On the path to becoming water positive, we
invested in 21 water replenishment projects that are expected to
generate 
over 1.3 million cubic meters of volumetric benefits in nine
water basins around the world. Progress toward our zero waste

commitment included diverting more than 15,200 metric tons of solid
waste otherwise headed to landfills and incinerators, 
as well as
launching new Circular Centers to increase reuse and reduce e-waste at
our datacenters.  
We contracted to protect over 17,000 acres of land
(50% more than the land we use to operate), thus achieving our

Total chunks: 344


### SentenceTransformersTokenTextSplitter

In [15]:
token_splitter = SentenceTransformersTokenTextSplitter(chunk_overlap=0, tokens_per_chunk=256)

token_split_texts = []
for text in character_split_texts:
    token_split_texts += token_splitter.split_text(text)

print(word_wrap(token_split_texts[10]))
print(f"\nTotal chunks: {len(token_split_texts)}")

increased, due in large part to significant global datacenter
expansions and the growth in xbox sales and usage. despite these
increases, we remain dedicated to achieving a net - zero future. we
recognize that progress won ’ t always be linear, and the rate at which
we can implement emissions reductions is dependent on many factors that
can fluctuate over time. on the path to becoming water positive, we
invested in 21 water replenishment projects that are expected to
generate over 1. 3 million cubic meters of volumetric benefits in nine
water basins around the world. progress toward our zero waste
commitment included diverting more than 15, 200 metric tons of solid
waste otherwise headed to landfills and incinerators, as well as
launching new circular centers to increase reuse and reduce e - waste
at our datacenters. we contracted to protect over 17, 000 acres of land
( 50 % more than the land we use to operate ), thus achieving our

Total chunks: 349


## Adding Retriever

In [16]:
embedding_function = SentenceTransformerEmbeddingFunction()

In [17]:
print(embedding_function([token_split_texts[10]]))

[array([ 4.25626226e-02,  3.32118124e-02,  3.03401463e-02, -3.48665826e-02,
        6.84165135e-02, -8.09091106e-02, -1.54743111e-02, -1.45090523e-03,
       -1.67444386e-02,  6.77077025e-02, -5.05414009e-02, -4.91953939e-02,
        5.13999686e-02,  9.19272453e-02, -7.17784464e-02,  3.95196751e-02,
       -1.28334910e-02, -2.49474477e-02, -4.62286361e-02, -2.43575070e-02,
        3.39497030e-02,  2.55024303e-02,  2.73171961e-02, -4.12621256e-03,
       -3.63383293e-02,  3.69089562e-03, -2.74304431e-02,  4.79672197e-03,
       -2.88962573e-02, -1.88707039e-02,  3.66662927e-02,  2.56958231e-02,
        3.13128568e-02, -6.39343858e-02,  5.39440587e-02,  8.22534710e-02,
       -4.17568088e-02, -6.99582696e-03, -2.34860461e-02, -3.07479613e-02,
       -2.97916541e-03, -7.79094100e-02,  9.35311429e-03,  3.16285598e-03,
       -2.22570635e-02, -1.82946678e-02, -9.61243641e-03, -3.15068662e-02,
       -5.51971328e-03, -3.27030569e-02,  1.68029815e-01, -4.74596359e-02,
       -5.00168130e-02, 

In [18]:
chroma_collection = chroma_client.create_collection(
    "microsoft_annual_report_2022", 
    embedding_function=embedding_function
)

In [19]:
ids = [str(i) for i in range(len(token_split_texts))]

chroma_collection.add(ids=ids, documents=token_split_texts)
chroma_collection.count()

349

## Querying Retriever

In [20]:
query = "What was the total revenue?"

In [21]:
results = chroma_collection.query(query_texts=[query], n_results=5)
retrieved_documents = results['documents'][0]

for document in retrieved_documents:
    print(word_wrap(document))
    print('\n')

74 note 13 — unearned revenue unearned revenue by segment was as
follows : ( in millions ) june 30, 2022 2021 productivity and business
processes $ 24, 558 $ 22, 120 intelligent cloud 19, 371 17, 710 more
personal computing 4, 479 4, 311 total $ 48, 408 $ 44, 141 changes in
unearned revenue were as follows : ( in millions ) year ended june 30,
2022 balance, beginning of period $ 44, 141 deferral of revenue 110,
455 recognition of unearned revenue ( 106, 188 ) balance, end of period
$ 48, 408 revenue allocated to remaining performance obligations, which
includes unearned revenue and amounts that will be invoiced and
recognized as revenue in future periods, was $ 193 billion as of june
30, 2022, of which $ 189 billion is related to the commercial portion
of revenue. we expect to recognize approximately 45 % of this revenue
over the next 12 months and the remainder thereafter. note 14 — leases


that are not sold separately. • we tested the mathematical accuracy of
management ’ s calculat

## Building RAG

In [23]:
def rag(query, retrieved_documents, model="gpt-3.5-turbo"):
    information = "\n\n".join(retrieved_documents)

    messages = [
        {
            "role": "system",
            "content": """You are a helpful expert financial research assistant. Your users are asking questions about 
            information contained in an annual report. You will be shown the user's question, and the relevant information 
            from the annual report. Answer the user's question using only this information."""
        },
        {
            "role": "user", 
            "content": f"Question: {query}. \n Information: {information}"
        }
    ]
    
    response = openai_client.chat.completions.create(
        model=model,
        messages=messages,
    )

    content = response.choices[0].message.content
    return content

In [24]:
query = "What was the total revenue?"

output = rag(query=query, retrieved_documents=retrieved_documents)

print(word_wrap(output))

The total revenue for the year ended June 30, 2022, was $198,270
million.
